<a href="https://colab.research.google.com/github/junjunmeng/Data-structure-and-Algo/blob/main/Take_home_Assignment_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Table of Contents

*  Initial Data Analysis
*  Data Wrangling
*  Exploratory Data Analysis
*  Statistical Analysis
*  Machine Learning





In [ ]:
pip install etsy_py

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pip install --upgrade pandas_profiling
import pandas_profiling as pp
# import etsy_py
from scipy.stats import shapiro
from scipy.stats import skew
from scipy.stats import kurtosis

from sklearn.cluster import DBSCAN
import sklearn as sklearn
from sklearn.utils import resample
from sklearn import preprocessing as preprocessing
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, accuracy_score, mean_squared_error
from sklearn.metrics import precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm
import numpy as np

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# <font color = 'blue'> 1. Data Clean

#### Initial Analysis

In [ ]:
def initial_analysis(df):
    """
    Given a dataframe produces a simple report on initial data analytics
    Params:
        - df 
    Returns:
        - Shape of dataframe records and columns
        - Columns and data types
    """
    print('Report of Initial Data Analysis:\n')
    print(f'Shape of dataframe: {df.shape}')
    print(f'Features and Data Types: \n {df.dtypes}')
    print("DataFrame Row Number: ", df.shape[0])
    print("Unique IDs: ", df.ID.nunique())

#### Percentage of missing value

In [ ]:
def percent_missing(df):
    """
    Given a dataframe it calculates the percentage of missing records per column
    Params:
        - df
    Returns:
        - Dictionary of column name and percentage of missing records
    """
    col=list(df.columns)
    perc=[round(df[c].isna().mean()*100,2) for c in col]
    miss_dict=dict(zip(col,perc))
    return miss_dict

#### Missing value exploration

In [ ]:
# missing values in a column, especially response variable
def missing_class(df, col_name):
  """
      Given a dataframe and colume, it calculates the count of missing records
    Params:
        - df: dataframe
        - col_name : col_name
    Returns:
        - number of records
  """
  missing_vals = ["No idea", np.nan, "#", "?"]
  print("Number of missing rows within " + col_name +" :"  + str(df[col_name].isnull().sum(axis=0)))

  # replace missing or abnormal value with np.nan
  df[col_name] = df[col_name].replace(missing_vals, np.nan)

  # dropping na
  df.dropna(subset = [col_name], inplace=True)
  
  # check the cleaned colname
  print(df.groupby(col_name).size())


#### Converting data types for selected columns

In [ ]:
# covert dataframe columns to numeric or other types

def convert_num(df, columns):
  """
      Given a dataframe and colume, it convert to another data type
    Params:
        - df: dataframe
        - columns : list of columns
    Returns:
        - dataframe with converted data type
  """
  for i in range(0, len(columns)):
    df[columns[i]] = df[columns[i]].astype("int64")

    

#### Duplicate check, basically two situations
1. duplicated identical rows
2. same ID with different values

In [ ]:
# any duplicate rows? especially in ID

def duplicate_row_remove(df, col_name):
  """
      Given a dataframe and colume, remove duplicated rows
    Params:
        - df: dataframe
        - col_name(str): a column name, e.g "ID"
    Returns:
        - number of remaining rows
  """
  global breast # dataframe name
  # original length of dataframe
  original_length = len(df)
  # Number of unique IDs
  print("Number of unique IDs: " + str(df[col_name].nunique()))
  # remove duplicated rows
  df = df[~df.loc[:, col_name:].duplicated()]

  # new length of dataframe
  new_length = len(df)
  # count of rows removed
  rows_removed = original_length - new_length
  print("Number of identical replicated rows should removed: " + str(rows_removed))
  print("Number of remaining rows: " + str(len(df)))
  breast = df

In [ ]:
# same user ID with different values

def duplicate_ID(df, col_name):
  """
      Given a dataframe and colume, remove non-identical rows with same ID
    Params:
        - df: dataframe
        - col_name: a column name, e.g ID
    Returns:
        - number of remaining rows
  """
  global breast # dataframe name
  ID_dup = df[df[col_name].duplicated()]
  print("Number of duplicated ID: " + str(len(ID_dup)))

  # keep the smallest index for each user
  df = df.sort_values(by= [col_name, "Index"], ascending = True)
  df = df[~df[col_name].duplicated(keep = 'first')]
  print("Number of remaining rows: " + str(len(df)))
  breast = df

#### Check and remove out-ranged Value

In [ ]:
def Incorrect_feature_values(df,columns, val, col_name ):
  """
      Given a dataframe and value, remove col_name's value > val
    Params:
        - df(dataframe): dataframe
        - col_name (str): a column name, e.g "ID"
        - val(int): specific value 
        - columns(list): columns list that need to check the value range
    Returns: dataframe removed rows with out-ranged value
  """
  global breast
  rows_with_large_vals = []
  for col in range(0, len(columns)):
    filter_by_col = df[columns[col]] > val
    ID_vals = pd.array(df[filter_by_col][col_name])
    rows_with_large_vals.append(ID_vals)

  # group list of lists into 1 list
  flat_list = []
  for sublist in rows_with_large_vals:
    for item in sublist:
      flat_list.append(item)
  
  # list of users with out range values
  users_with_large_vals = pd.array(flat_list).unique()
  print("There are "+ str(len(users_with_large_vals)) + " users with values that exceed " + str(val))

  # remove record with any out-range values
  df = df[~df[col_name].isin(pd.array(users_with_large_vals))]
  breast = df

#### Re-assign value to a column

In [ ]:
# rename the class based on code
def reClass(df, col_name):
  """
      Given a dataframe and col_name, re-assign value according to multiple conditions
    Params:
        - df(dataframe): dataframe
        - col_name (str): a column name, e.g "ID"
    Returns: dataframe's column with re-assigned value
  """
  global breast
  class_new = []
  for item in df[col_name]:
    if item == 2:
      class_new.append(0)
    elif item ==  4:
      class_new.append(1)
    else:
      class_new.append(np.nan)
  #df.drop(df_col)
  df['Class_new'] = class_new
  # drop original column
  df = df.drop(col_name, axis = 1)
  # assign new column as original column name
  df = df.rename(columns = {'Class_new': col_name})
  # remove NA
  df = df[df[col_name] != np.nan]
  breast = df


#### Aggregation summary of each columns in a dataframe

In [ ]:
# check the aggregation for each column
def col_agg(df, col_list):
  col_summary = []
  for name in col_list:
    col_summary.append(df.groupby(name).size().reindex())
  return col_summary

# eg.  col_agg(breast, breast.columns[1:12])

# <font color = 'blue'> 2. Data Exploratory Analysis

#### Set Profile Report

In [ ]:
def profile(df):

  """
    Given a dataframe, return data profile 
    Params:
      - df(dataframe): dataframe
    Returns: data profile in html format
   """
  global prof
  from pandas_profiling import ProfileReport
  prof = ProfileReport(df)
  prof.to_file(output_file= "data_profile.html")
  return prof

#### Count plot in Seaborn

In [ ]:
def sub_countplot(df, Class):
  """
    Given a dataframe, return data profile 
    Params:
      - df(dataframe): dataframe
      - Class(str): col_name, e.g "Class", usually response variable
    Returns: count_plot by features
  """ 
  features = df.columns[1:-1].to_list()
  feature_num = len(features)
  x = 3
  y = feature_num//3 + feature_num%3
  fig, ax = plt.subplots(x, y, figsize= (15,15))
  for i in range(feature_num):
    sns.countplot(x= features[i], hue= Class, data = df, ax = ax[i//3, i%3])


#### Check normality

In [ ]:
def normality_test(df,col_list):
    """
    Given a dataframe determines whether each numerical column is Gaussian 
    H0 = Assumes distribution is not Gaussian
    Ha = Assumes distribution is Gaussian
    Params:
        - df
    Returns:
        - W Statistic
        - p-value
        - List of columns that do not have gaussian distribution
    """
    non_gauss=[]
    w_stat=[]
    # Determine if each sample of numerical feature is gaussian
    alpha = 0.05
    for n in col_list:
        stat,p=shapiro(df[n])
        print(sns.distplot(df[n]))
        print(n, "skew is:", skew(df[n]), "kurtosis is :", kurtosis(df[n]))


        if p <= alpha: # Reject Ho -- Distribution is not normal
            non_gauss.append(n)
            w_stat.append(stat)
    # Dictionary of numerical features not gaussian and W-Statistic        
    norm_dict=dict(zip(non_gauss,w_stat))
    return norm_dict

#### Outliers by Boxplot

In [ ]:
# Outliers by boxplot
col_names = breast.columns[2:11]
col_names

breast.boxplot(column=['Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
       'Normal Nucleoli','Mitoses' ], grid=False, rot=90, fontsize=11)


#### Data transformation by boxcox

In [ ]:
# data transformation?
from scipy.stats import boxcox
plt.hist(boxcox(breast['Clump Thickness'],1))



#### Visualize the binary classes count

In [ ]:
def class_fig(col):
  """
  Given the response variable, it visulize the count in binary classes
  Params:
    - Series
  Returns:
    - countplot
  """
  sns.countplot(col, label = 'Count')
  plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

#### Correlation heatmap

In [ ]:
# correlation between features

def corr_heatmap(df, col_names):
  corr = df[col_names].corr()
  mask = np.zeros_like(corr)
  mask[np.triu_indices_from(mask)] = True
  sns.heatmap(corr, annot= True, vmin= -1, vmax= 1, mask= mask )
  plt.show()


#### Feature Importance by Random Forest

In [ ]:
# feature importance by Random Forest

def rf_importance(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state = 42)
  rf = RandomForestClassifier()
  rf.fit(X_train,y_train)
  
  # Feature Importance
  importance_rf = pd.Series(rf.feature_importances_, index = X_train.columns)
  sorted_importance_rf = importance_rf.sort_values()
  sorted_importance_rf.plot(kind = 'barh', color = 'lightgreen')
  plt.title("Feature Importance by Random Forest")
  plt.show()

# <font color = 'blue'> 3. Modeling

#### SMOTE for resampling

In [ ]:
# SMOTE for Upsampling training 

from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split( breast[breast.columns[0:8]], breast['Class'], test_size= 0.3, random_state = 42)
X_res, y_res = SMOTE(random_state = 42).fit_sample(X_train, y_train)

#### Normalization

In [ ]:
X_train = pd.DataFrame(preprocessing.normalize(X_train))
X_test = pd.DataFrame(preprocessing.normalize(X_test))

#### Standardlization


In [ ]:
# standardlization

ss = StandardScaler()
train_X = ss.fit_transform(train_X)
test_X = ss.fit_transform(test_X)

#### Logistic Regression with sklearn

In [ ]:
# Logistic Regression 

def LogReg(X_res, y_res, X_test, y_test):
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import classification_report, confusion_matrix
  from sklearn.metrics import roc_auc_score
  from sklearn.metrics import roc_curve
  logreg = LogisticRegression().fit(X_res, y_res)
  y_pred = logreg.predict(X_test)
  print("confusion_matrix:\n", confusion_matrix(y_test, y_pred))
  print("Classification report:\n", classification_report(y_test, y_pred))
  print("ROC:", roc_auc_score(y_test, y_pred))

  # compute predicted probabilites: y_pred_prob
  y_pred_prob = logreg.predict_log_proba(X_test)[:, 1]

  # generate ROC curve

  fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

  # plot ROC curve

  plt.plot([0,1], [0,1], 'k--')
  plt.plot(fpr, tpr)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC Curve')
  plt.show()


#### Logistic Regression with statsmodels.api

In [ ]:
def logit_SM(X_train, y_train):
  import statsmodels.api as sm
  logit_model = sm.Logit(y_train.values.ravel(), X_train)
  result = logit_model.fit()
  print(result.summary2())

#### Random Forest

In [ ]:
# random Forest
def RF_pipe(X_res, y_res, X_test, y_test):
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.metrics import classification_report, confusion_matrix
  from sklearn.metrics import roc_auc_score
  from sklearn.metrics import roc_curve
  rf = RandomForestClassifier().fit(X_res, y_res)
  y_pred = rf.predict(X_test)
  print("confusion_matrix:\n", confusion_matrix(y_test, y_pred))
  print("Classification report:\n", classification_report(y_test, y_pred))
  print("ROC:", roc_auc_score(y_test, y_pred))




# <font color = 'green'> Example 1: Conversion rate
The goal of this challenge is to build a model that predicts conversion rate and, based on the model, come up with ideas to improve it.

We have data about all users who hit our site: whether they converted or not as well as some of their characteristics such as their country, the marketing channel, their age, whether they are repeat users and the number of pages visited during that session (as a proxy for site activity/time spent on site).

Your project is to:

1.   Predict conversion rate
2.   Come up with recommendations for the product team and the marketing team to improve conversion rate
 


## Data Exploration with Seaborn
[Seaborn Gallary](https://seaborn.pydata.org/examples/index.html)